In [ ]:
import pandas as pd

In [ ]:
input_path = '../tmp-data/ss_range_bars.sell_above_adv'
known_false_signals = False


In [ ]:
df = pd.read_csv(f'{input_path}.csv', parse_dates=['timestamp'], index_col='timestamp')
df

In [ ]:
def get_start_end_idx_for_period(df: pd.DataFrame, resample_arg: str = 'M') -> str:
        # resample the dataframe into monthly periods
        groups = df.resample(resample_arg)
        # create a list of tuples containing the start and end indices for each period
        start_end_indices = []
        # iterate over each group (period) in the resampled DataFrame
        for period_start, group_df in groups:
            # get the start and end indices of the rows within the current period
            period_start = group_df.index[0]
            period_end = group_df.index[-1]
            # add the start and end indices to the list
            start_end_indices.append((period_start, period_end))
        return start_end_indices

In [ ]:
start_end_indices = get_start_end_idx_for_period(df)
start_end_indices

In [ ]:
from backtesting import Strategy
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

class RangeBarStrategyPreCalc(Strategy):

    per_trade_risk = 0.06
    
    def init(self):
       self.false_buys = 0
       self.false_sells = 0

    def next(self):
        current_close = self.data.Close[-1]
        range_size = self.data.average_adr[-1] * 0.1
        potential_profit = self.data.average_adr[-1] * 0.15
        
        pre_calc_signal = self.data.signal[-1]
        pre_calc_false_signal = self.data.false_signal[-1]
    
        known_false_pass = True
        if known_false_signals:
            known_false_pass = pre_calc_false_signal != 1

        if pre_calc_signal == 1 and known_false_pass:
            if pre_calc_signal == 1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal buy')
                    self.false_buys += 1
            else:
                print(f'ERROR: buying, against pre_calc_signal')    
            self.buy(size=self.per_trade_risk, sl=current_close - range_size, tp=current_close + potential_profit)
        elif pre_calc_signal == -1 and known_false_pass:
            if pre_calc_signal == -1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal sell')
                    self.false_sells += 1  
            else:
                print(f'ERROR: selling, against pre_calc_signal')
            self.sell(size=self.per_trade_risk, sl=current_close + range_size , tp=current_close - potential_profit)

In [ ]:
def backtest_period(strat, df, all_stats, start, end, cash=100000, leverage = 20):
    from backtesting import Backtest
    # from time import sleep 
    bt = Backtest(df, strat, cash=cash, margin=1/leverage, commission=(0.03 / 100), exclusive_orders=False)
    stats = bt.run()
    stats_cp = stats.copy()
    stats_cp['start'] = start
    stats_cp['end'] = end
    all_stats.append(stats_cp)
    # sleep(3)
    false_buys = stats._strategy.false_buys
    false_sells = stats._strategy.false_sells
    trades = stats['# Trades']
    return_percentage = stats['Return [%]']
    win_rate_percentage = stats['Win Rate [%]']
    profit_factor = stats['Profit Factor']
    net_profit = (stats['Equity Final [$]'] - cash)
    equity = stats['Equity Final [$]']
    max_drawdown_percentage = stats['Max. Drawdown [%]']
    average_drawdown_percentage = stats['Avg. Drawdown [%]']
    percentage_false_trades = round(((false_buys + false_sells) / int(trades)) * 100, 2)
    return {
        "net_profit": net_profit,
        "start": str(start),
        "end": str(end),
        "return_percentage": return_percentage,
        "win_rate_percentage": win_rate_percentage,
        "profit_factor": profit_factor,
        "max_drawdown_percentage": max_drawdown_percentage,
        "average_drawdown_percentage": average_drawdown_percentage,
        "trades": trades,
        "false_buys": false_buys,
        "false_sells": false_sells,
        "percentage_false_trades": percentage_false_trades,
        "equity": equity
    }

errors don't seem to occur on a particular period, as demonstrated by the following code:

In [ ]:
# def test_period(idx: int):
#     import json
#     start, end = start_end_indices[idx]
#     print(f'start: {str(start)}, end: {str(end)}')
#     result = backtest_period(RangeBarTrailingStrategyPreCalc, df, start, end)
#     print(json.dumps(result, indent=4))

# test_period(1)    

In [ ]:

def test_periods(start_end_indices, all_stats):
    # import json
    profit_takeout = []
    bulk_stats = []
    errors = []
    current_equity = 100000
    initial_equity = 100000
    for start, end in start_end_indices:
        print(f'start: {str(start)}, end: {str(end)}')
        try:
            df_sample = df[start:end].copy()    
            result = backtest_period(RangeBarStrategyPreCalc, df_sample, all_stats, start, end, cash=current_equity)
            if result['equity'] < initial_equity:
                current_equity = result['equity']
            else:
                current_equity = initial_equity
                takeout = result['equity'] - initial_equity
                profit_takeout.append([f'{start}-{end}', takeout / 100])
            # print(json.dumps(result, indent=4))
            bulk_stats.append(result)
        except Exception as e:
            errors.append(f'strat: RangeBarStrategyPreCalc,  start: {str(start)}, end: {str(end)}. error: {str(e)}')

    print(f'errors: {len(errors)}')
    print(f'errors: {errors}')
    df_stats = pd.DataFrame(bulk_stats)
    return df_stats, all_stats, profit_takeout


df_stats, all_stats, profit_takeout = test_periods(start_end_indices, [])
takeout = [x[1] for x in profit_takeout]
profit_takeout, sum(takeout)


### sell only volume above limit (535.77 for buy also -- bad idea)

* 873,656.46 volume above 35 @ 7% per trade
* 873,147.50 volume above 0 @ 7% per trade
* 882,868.51 volume above 50 @ 7% per trade
* 946,809.30 volume above 100 @ 7% per trade
* 1,061,383.93 volume above 200 @ 7% per trade
* 6,604,260.28 volume above adv @ 10% per trade
* 22,767.65 volume above adv @ 2% per trade
* 317,835.40 volume above adv @ 5% per trade
* 637,770.41 volume above adv @ 6% per trade

In [ ]:
print(f'all_stats: {len(all_stats)}')
all_stats_str = ""
for stats in all_stats:
    all_stats_str += stats.to_string() + f'\n{stats._trades.to_string()}\n' + "\n\n-------------------------------------------------------------------------------------------------\n\n"

suffix = input_path.split('/')[-1].split('.')[-1]
with open(f'../tmp-data/all_stats-{suffix}.txt', 'w') as f:
    f.write(all_stats_str)    

Using 100_000 due to limitations of backtesting.py, so we divide the net profit sum by 100_000, to see what we'd get it we started with 1_000 

In [ ]:
net_profit_sum = df_stats['net_profit'].sum() / 100000
net_profit_sum

Starting with 1,000 USDT 100% reinvest


* no false signals: net_profit_sum: 226,864.80
* with false signals: net_profit_sum: 12,623.53
* with false signals & ant-squeeze 10: net_profit_sum: 17,942.44
* with false signals & ant-squeeze 10 & without rb bug: net_profit_sum: 39304.98

Starting with 1,000 USDT 75% reinvest

* no false signals: net_profit_sum: 9,559.34
* with false signals: net_profit_sum: 605.87

margin 1/20 tot pp: 903.04 (false)  
margin 1/20 tot pp: 10274.68 (no false)  

how to stop losses from large drawdown ?

In [ ]:
pf_measure = df_stats['profit_factor'].sum()
pf_measure

In [ ]:
dd_p_measure = df_stats['max_drawdown_percentage'].sum()
dd_p_measure

## using sell if is_volume_above_adv_limit
### > row['adv'] * 0.75 @ 7% per trade
* dd_p_measure: -280.605
* pf_measure: 32.75
## > row['adv'] @ 10% per trade
* dd_p_measure: -320.35
* pf_measure: 37.74
## > row['adv'] @ 5% per trade
* dd_p_measure: -184.59
* pf_measure: 37.76
## > row['adv'] @ 2% per trade
* dd_p_measure: -70.12
* pf_measure: 36.86

In [ ]:
sorted_df_stats = df_stats.sort_values(by='profit_factor')
sorted_df_stats 

In [ ]:
if known_false_signals:
    sorted_df_stats.to_csv(f'{input_path}-monthly-stats_no-false-signals.csv')
else:
    sorted_df_stats.to_csv(f'{input_path}-monthly-stats.csv')